In [1]:
from lale.datasets.openml import fetch

In [2]:
(X_train, y_train), (X_test, y_test) = fetch("jungle_chess_2pcs_raw_endgame_complete", "classification")

In [3]:
import ray
ray.shutdown()

In [4]:
# from lale.helpers import cross_val_score

In [5]:
# sk_pipeline = pipeline.export_to_sklearn_pipeline()

In [6]:
# sk_pipeline

In [7]:
import ray
ray.init(object_store_memory=16 * 1024 * 1024 * 1024)

2021-05-26 11:34:55,429	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8267


{'node_ip_address': '9.211.114.84',
 'raylet_ip_address': '9.211.114.84',
 'redis_address': '9.211.114.84:20725',
 'object_store_address': '/tmp/ray/session_2021-05-26_11-34-53_711763_35035/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-05-26_11-34-53_711763_35035/sockets/raylet',
 'webui_url': '127.0.0.1:8267',
 'session_dir': '/tmp/ray/session_2021-05-26_11-34-53_711763_35035',
 'metrics_export_port': 60055,
 'node_id': '6f77fe1897704e0f2f8967852a1ccd077086efebfb0634ec3ca3d130'}

In [8]:
import codeflare.pipelines.Datamodel as dm

In [9]:
from sklearn.model_selection import KFold, StratifiedKFold

In [10]:
kf = StratifiedKFold(n_splits=10)

In [11]:
pipeline = dm.Pipeline()

In [12]:
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import Nystroem
from sklearn.feature_selection import SelectKBest

In [13]:
feature_union = FeatureUnion(transformer_list=[('PCA', PCA()),
                                                ('Nystroem',
                                                 Nystroem()),
                                                ('SelectKBest',
                                                 SelectKBest(k=3))])

In [14]:
random_forest = RandomForestClassifier(n_estimators=200)

In [15]:
node_fu = dm.EstimatorNode('feature_union', feature_union)
node_rf = dm.EstimatorNode('randomforest', random_forest)

In [16]:
pipeline.add_edge(node_fu, node_rf)

In [17]:
import codeflare.pipelines.Runtime as rt
from codeflare.pipelines.Runtime import ExecutionType

In [18]:
pipeline_input = dm.PipelineInput()
xy = dm.Xy(X_train, y_train)
pipeline_input.add_xy_arg(node_fu, xy)

In [19]:
%%time
scores = rt.cross_validate(kf, pipeline, pipeline_input)

CPU times: user 3.39 s, sys: 1.64 s, total: 5.02 s
Wall time: 1min 22s


In [21]:
scores

[0.8185148185148186,
 0.8175158175158175,
 0.8128538128538129,
 0.8195138195138195,
 0.8228438228438228,
 0.8168498168498168,
 0.8131868131868132,
 0.8161838161838162,
 0.7991339107261826,
 0.8077948034643571]

In [24]:
from lale.lib.sklearn import PCA, Nystroem, SelectKBest, RandomForestClassifier
from lale.lib.lale import ConcatFeatures

pipeline = (PCA() & Nystroem() & SelectKBest(k=3)) >> ConcatFeatures() >> RandomForestClassifier(n_estimators=200)
# pipeline.visualize()

In [25]:
%%time
from lale.helpers import cross_val_score
cross_val_score(pipeline, X_train, y_train, cv=10)

CPU times: user 7min 39s, sys: 13.1 s, total: 7min 52s
Wall time: 7min 13s


[0.8185148185148186,
 0.8101898101898102,
 0.8148518148518149,
 0.8201798201798202,
 0.8145188145188145,
 0.8185148185148186,
 0.8168498168498168,
 0.8125208125208125,
 0.7978014656895404,
 0.8114590273151232]